# tileSetter Testing & Process



In [6]:
from pyleSetter import *

Updating stuck name prefixes ...
A markers tagged: 7
B markers tagged: 6
C markers tagged: 7
D markers tagged: 3
F markers tagged: 4
G markers tagged: 2
J markers tagged: 1
K markers tagged: 5
L markers tagged: 7
M markers tagged: 7
N markers tagged: 2
P markers tagged: 6
R markers tagged: 4
S markers tagged: 14
T markers tagged: 6
Active prefixes updated!

Names Updated!
Generator
------------------
  pieces Lmin Lmax              regex
0   bit*    3    5  [a-z][a-z][a-z]..
1   bit*    2    3        [a-z][a-z].
------------------

Name Stats
Stuck? True
--------------
           NUM
5 Letters   75
6 Letters   76
7 Letters   76
8 Letters   75
           ---
All Names  302
--------------
Last Update @ 09:50:54 PM

Loaded name generator 'bb'.Open /HTML/tileSetter.html in your favorite browser to see the tiles!
To generate more names, type 'refresh(bb)')
or create new name generator using nb.nameGen().


In [28]:
#newbie = nb.w_s.loc[:,nb.w_s.columns!='cvc']
#newbie.to_csv('./data/word-segments.csv',in)
#'dragon'[0:3]
w_s2 = update_pres(nb.w_s)

A markers tagged: 7
B markers tagged: 6
C markers tagged: 7
D markers tagged: 3
F markers tagged: 4
G markers tagged: 2
J markers tagged: 1
K markers tagged: 5
L markers tagged: 7
M markers tagged: 7
N markers tagged: 2
P markers tagged: 6
R markers tagged: 4
S markers tagged: 14
T markers tagged: 6
Active prefixes updated!


In [29]:
test = w_s2[w_s2.prefix==True]
test[test.L==3]

,bit,L,prefix
84331,beo,3.0,True


In [6]:
class Tagger:
    def __init__(self, tag, single=False,**attrs):
        self.tag = tag
        self.ats = self.check_for_class(attrs)
        self.single = single

        if single==True:
            self.openr = ' />'
            self.tagclosed = ''
        else:
            self.openr = '>'
            self.tagclosed = '</' + tag + '>'

    def check_for_class(self,attributes):
        if 'class_' in attributes.keys():
            attributes['class'] = attributes.pop('class_')
        return attributes

    def wrap(self, to_wrap, wrap_by='lines', **attrs):
        attrs = self.check_for_class(attrs)
        #print(attrs)
        tagclosed = self.tagclosed

        if wrap_by == 'lines':
            wrapper = pd.DataFrame()
            tagopen = self.open_tag(attrs.copy())
            #print(tagopen)


            if type(tagopen) == pd.Series:
                wrapper['open'] = tagopen
            elif type(tagopen) == str:
                open = pd.Series(tagopen,index = range(0,len(to_wrap)))
                wrapper['open'] = open

            wrapper['content'] = '\t' + to_wrap
            wrapper['closed'] = tagclosed
            #print(wrapper.stack().explode().swaplevel(0,1))
            wrapper['tag'] = wrapper.open.astype(str) + \
                             wrapper.content.astype(str) + \
                             wrapper.closed.astype(str)
            wrapped = wrapper['tag']#wrapper['tag']


        elif wrap_by == 'block':

            content = to_wrap
            if type(to_wrap) == list:
                content = pd.Series(to_wrap)
            if type(content) == pd.Series:
                content = '\t'+content
                content = content.values.tolist()
            elif type(content) == str:
                content = [ '\t'+content ]
            else:
                print('Unable to block wrap.')
                return

            wrapper = [ self.open_tag(attrs.copy()) ] + content + [ tagclosed ]
            wrapped = pd.Series(wrapper)

        return wrapped

    def open_tag(self,attrs):

        ats = self.ats.copy()
        ats.update(attrs.copy())
        tag_open = '<' + self.tag
        for attribute,value in ats.items():
            if type(value) == str:
                tag_open += ' {}="{}"'.format(attribute,value)
            elif type(value) == pd.Series:

                tag_open += ' ' + attribute + '="' + value + '"'
        tag_open += self.openr

        return tag_open

    def add_attributes(self,**attrs):
        check = self.check_for_class(attrs)
        self.ats.update(check)

    def copy(self):
        new_boi = self
        return new_boi


a = Tagger('a', href='')
img = Tagger('img', single=True, src='' )
div = Tagger('div')
p = Tagger('p')
h2 = Tagger('h2')
h1 = Tagger('h1')
br = Tagger('br',single=True)

In [9]:
def unpack_HTML(unpack_me):
        # unpacks HTML from dictionary into pandas series
        page = []
        for keys,values in unpack_me.items():
            page += values.tolist()
            page += ['\n']

        page = pd.Series(page)
        return page


def get_pet_links(names,img_no='1'):

    # image URL
    names['img'] = 'http://pets.neopets.com/cpn/' +\
                    names.Neopet + '/1/'+img_no+'.png'
    
    # URLs to important places
    names['petpage'] = 'http://neopets.com/~' + names.Neopet
    names['pound'] = 'http://neopets.com/pound/adopt.phtml?search=' + names.Neopet
    names['lookup'] = 'http://neopets.com/petlookup.phtml?pet=' + names.Neopet

    return names


def get_pet_tiles(names,link_to='petpage',tiyg=False):
    
    # Create tiles
    ps = div.wrap(p.wrap(names.Neopet),class_="pet",
                    style = "background: url('"+ names.img + "');")

    # Make tiles circles if updating Tiyg's petpage
    a_style = ''
    if tiyg == True:
        a_style = "border-radius:80px;"

    # Add pet tile HTML column to 'names'
    names['pet_div'] = a.wrap(  ps,href=names[link_to],
                                style=a_style )
    return names

def make_L_divs(names):
        divdict = {}
        for i in names.L.unique():
            nameslice = names[names.L == i]

            i_name = str(i) + ' Letters'
            iL_div =  [ h2.wrap(i_name,wrap_by='block') ]
            iL_div += nameslice.pet_div.values.tolist()
            iL_div = pd.Series(iL_div)
            Ldiv = div.wrap(iL_div,wrap_by='block',class_='petgroup')

            divdict[i_name] = Ldiv.explode()
        return divdict

def tileSet(names,linkto='petpage'):
    css = '<link rel="stylesheet" type="text/css" href="style.css">'
    css = pd.Series(css)
    sort_pets_by = {'by':['L','Neopet']}
    
    names0 = names.copy()
    names0['Neopet'] = names0.Neopet.str.replace('~','_')
    names1 = get_pet_links(names0, img_no='1') # get pet links
    names2 = get_pet_tiles(names1, link_to=linkto) # get pet tile HTML
    
    page = {'css': css}
    
    divs = make_L_divs(names2) # sort pet tiles into divs based on name length
    
    page.update(divs)
    
    html = unpack_HTML(page)
    
    html.to_csv('./HTML/tileSet.html', sep=',', header=False,
                index=False, quoting=csv.QUOTE_NONE,
                escapechar='\n')
    
    print('HTML Updated!')
    return

## Word Segment Loading &  Setup

In [27]:
#--! Import New Segments ------------------------------------------ #
def new_segments(filepath,old_df=''):
    # Imports a csv file containing a list of word/name new_segments
    # Integrates segments into existing segment data

    segments = pd.read_csv(filepath)
    print('New word segments loaded.')
    clean_seg = segment_cleaner(segments)
    print('New word segments cleaned + determined length + cvc format')
    labeled = label_pres(clean_seg)
    print('New prefixes labeled.')

    input1 = input('Would you like to merge new segments with existing segments? y/n\n')
    if input1 == 'y':
        output = merge_segments(labeled,old_df)
        print('Merge accepted. Returning merged segment dataframe.')
    elif input1 =='n':
        print('Merge declined. Returning new segments only.')
        output = labeled
    else:
        print('Invalid input. Defaulting to return new segments only.')
    return output

#--! Merge new segments into old segments -------------------------- #
def merge_segments(new,old):
    if old == '':
        error('No existing word segment data found.')
    else:
        merged = pd.concat([new,old],ignore_index=True).sort_values(by='bit')
        merged = merged.drop_duplicates().reset_index(drop=True)

        input = ('Word segment data merged. Would you like to save? y/n')
        if input == 'y':
            fname = 'WS_' + str(dt.date.today) + '.csv'
            merged.to_csv('./data/word-segments/' + fname,index=False)
            merged.to_csv('./data/word-segments.csv',index=False)

    return merged

#--! Word Segment Cleaner ------------------------------------------- #
def segment_cleaner(df):
    # Takes in a series of word/name segments
    # Returns dataframe with segments + length + cvc format

    df = df.drop_duplicates().sort_values() # Drop duplicates
    df = df[~df.str.contains(r'[_0-9]')].reset_index(drop=True) # Remove non-letters

    Ls = df.str.len() # get length

    # Construct dataframe
    final = pd.DataFrame({})
    final.insert(loc=0,column='bit',value=df)
    final.insert(loc=1,column='L',value=Ls)
    return final

# #--! Label Segments As Prefixes ------------------------------------- #
def update_pres(bit_df, abcs=string.ascii_lowercase):

    stucks = pd.read_csv('./data/stuck-strings.csv')
    stucks = stucks.replace('\_','~',regex=True)
    stucks = stucks[~stucks.prefix.str.startswith('~')].reset_index(drop=True)
    markers = stucks.marker.str.lower()
    
    bit_df['prefix'] = False
    N_tagged = 1
    for n,i in enumerate(markers):
        if n>0 :
            if markers[n][0] != markers[n-1][0]:
                print(markers[n-1][0].upper()+' markers tagged: '+ str(N_tagged))
                N_tagged = 1
            elif n == len(markers)-1:
                N_tagged += 1
                print(markers[n-1][0].upper()+' markers tagged: '+ str(N_tagged))
            else:
                N_tagged += 1

        sliced = bit_df[bit_df.bit.str.startswith(i[0:3])].copy()
        # Get regex filter string
        F = get_regex(i)
        
        # Tag prefixes that match regex as 'True'
        sliced.loc[sliced.bit.str.match(F),'prefix'] = True
        
        bit_df.update(sliced)
    print('Active prefixes updated!')
    return bit_df

# def update_pres(bit_df, abcs=string.ascii_lowercase):

#     stucks = pd.read_csv('./data/stuck-strings.csv')
#     stucks = stucks.replace('\_','~',regex=True)
#     stucks = stucks[~stucks.prefix.str.startswith('~')].reset_index(drop=True)
#     markers = stucks.marker.str.lower()

#     print('Updating stuck name prefixes ...')
#     bit_df['prefix'] = False
#     N_tagged = 1
#     for n,i in enumerate(markers):
#         if n>0 :
#             if markers[n][0] != markers[n-1][0]:
#                 print(markers[n-1][0].upper()+' markers tagged: '+ str(N_tagged))
#                 N_tagged = 1
#             elif n == len(markers)-1:
#                 N_tagged += 1
#                 print(markers[n-1][0].upper()+' markers tagged: '+ str(N_tagged))
#             else:
#                 N_tagged += 1

#         sliced = bit_df[bit_df.bit.str.startswith(i[0:3])].copy()
#         # Get regex filter string
#         F = get_regex(i)

#         # Tag prefixes that match regex as 'True'
#         sliced.loc[sliced.bit.str.fullmatch(F),'prefix'] = True

#         bit_df.update(sliced)
#     print('Active prefixes updated!')
#     return bit_df

def get_regex(marker, abcs=string.ascii_lowercase):
    # Takes a "marker" string and generates regex
    # To select all prefixes in that group that
    # come alphabetically after that marker

    m = marker
    # go through marker backwards and omit first 2 letters of marker
    # e.g., dragon -> noga
    for i,L6 in enumerate(m[::-1][:-2]) :
        L5 = m[-i-2]

        if i == 0:
            if m[-1] == '~': # last character is underscore -> do not include last letter in regex selection
                R6 =''
                L5 = abcs[abcs.find(L5)+1]
            else:
                R0 = abcs[abcs.find(L6)]
                R6 = '['+ R0 + '-z]' # range of last letter
            F = L5 + R6

        else:
            if L6 == 'z':
                R0 = abcs[abcs.find(L6)] # avoid bounds errors when L6 = z
            else:
                R0 = abcs[abcs.find(L6)+1] # range starts at letter after L6
            R = '['+ R0 + '-z]'

            F = L5 +'('+R+'|'+F+')' # Add to existing regex


    F = m[0]+F+'.*' # add start indicator and .* to select full segment
    return F

# #--! Regex Generator for Prefix Filtering --------------------------- #
# def get_regex(marker):
#     # Takes a "marker" string and generates regex
#     # To select all prefixes in that group that
#     # come alphabetically after that marker

#     m = marker
#     # go through marker backwards and omit first 2 letters of marker
#     # e.g., dragon -> noga
#     for i,L6 in enumerate(m[::-1][:-2]) :
#         L5 = m[-i-2]

#         if i == 0:
#             if m[-1] == '~': # last character is underscore -> do not include last letter in regex selection
#                 R6 =''
#                 L5 = abcs[abcs.find(L5)+1]
#             else:
#                 R0 = abcs[abcs.find(L6)]
#                 R6 = '['+ R0 + '-z]' # range of last letter
#             F = L5 + R6

#         else:
#             if L6 == 'z':
#                 R0 = abcs[abcs.find(L6)] # avoid bounds errors when L6 = z
#             else:
#                 R0 = abcs[abcs.find(L6)+1] # range starts at letter after L6
#             R = '['+ R0 + '-z]'

#             F = L5 +'('+R+'|'+F+')' # Add to existing regex


#     F = m[0]+F+'.*' # add start indicator and .* to select full segment
#     return F

## Build Names From Matrix

In [754]:
def build_names( name_matrix, N, bits=w_s, stuck=True ):
    segments = pd.DataFrame(columns=name_matrix.piece,index=list(range(N)))
    ng = name_matrix
    for piece in ng.itertuples():
        # index = piece[0]
        # piece name = piece[1]
        L = piece[2]
        # regex = piece[3]
        
        
        if piece[1] == piece[3]:
            segments.iloc[:,piece[0]] = piece[1]
        else:
            samply = w_s[w_s.L==L]
            
            if stuck == True and piece[0] == 0:
                if L < 3:
                    print('WARNING: Stuck prefixes must have at least 3 letters! Generating non-sticky names.')
                    stuck=False
                samply = samply[samply.prefix==stuck]
                
                    
            
            samply = samply[samply.bit.str.match(piece[3])]
            
            if len(samply) < N:
                rep = True
            else:
                rep = False
            segments.iloc[:,piece[0]] = samply.bit.sample(n = N,replace=rep).reset_index(drop=True)
    names = pd.DataFrame(index=segments.index,columns=['Neopet','L'])
    names.Neopet = ''
    for i in range(len(segments.columns)):
            names.Neopet += segments.iloc[:,i]
    names.Neopet = names.Neopet.str.capitalize()
    names.L = names.Neopet.str.len()
    return names

## Name Generator
```python
nameGen( name_format, N=300, stuck=True, spread='equal' )
```

Creates a new Name Generator object.

### Parameters

####  `name_format` (*str*) :

Groups together "word segments" that provide a template from which to generate names.


#### `N` (*int, default 300*) :

Number of names this generator will generate by default. Can be over-ridden.

---
#### `name_format` Details :

| segment | Description |
|-:| :- |
| **`bit:LL:cvx_s`** | picks random word segments |
|  *`:LL`*   | Default: `16`<br>Minimum & max segment lengths |
|  *`:cvx_s`*| Default: `.`<br>Constrain word segment characters to certain consonants, vowels, or regex |
| **`cvx:L`** | picks random letter/consonant/vowel for each character |
| *`:L`* | Default: `1`<br>Repeat the preceeding letters `L` times |
|**`string`**| String of letters to be held constant among all generated names |

* Use `,` to join segments together. 
* Note that **`bit::`** and **`cvs:`** *MUST* include all colons.
* **``bit::``** and **``cvs:``** will use defaults if no values are specificed after the colons.
* For more details, see the page on [creating name format strings](link_eventually).

##### Examples:


| `name_format`   | Description  | Examples   | Resulting Names | 
| :-------------: | :----------- | :--------- | :-------------- |
|`'bit::,xcv:'`   | word segment, letter, consonant, vowel | `be,are`, `angel,ddo`|Beare, Angelddo | 
|`'doot,bit:13:'` | 'doot', 1L-3L word segment |`doot,n`, `doot,ezz`| Dootn, Dootezz | 
|`'v:3,bit:12:.v'`| 3 random vowels, random 1L-2L word segment |`yea,bo`, `ayo,r`|Yeabo, Ayor | 


In [3]:
#--! Name Generator class ----------------------------------------- #
build_names = nb.build_names
class nameGen:
    def __init__(self,name_format,N=300,stuck=True,spread='equal'):
        # Default Properties
        self.nf = name_format.lower()
        self.stuck = stuck
        self.N = N
        self.spread = spread

        # Properties to be calculated
        self.Lmin = 0
        self.Lmax = 0
        self.ng = [] # name generator
        self.ng_list = []
        self.names = []
        self.stats = {'time':'','format':'','name_data':''}

        # Run other functions
        self.get_ng()        # create name generator(s)
        self.get_names()     # generate names

    def ngs(self):
        for i in self.ng_list:
            for j in i:
                print(j)
                print('---')
    def update_stats(self):

        stats = {}
        for i in self.names.L.unique():
            ind = (str(i) + ' Letters')
            n = len(self.names[self.names.L == i])
            stats.update({ ind : n })
        stats.update({' ':'---',
                      'All Names':len(self.names)
                     })


        stats = pd.DataFrame.from_dict(stats,orient='index')
        stats.columns = [str('NUM')]

        self.stats['name_data'] = stats
        self.stats['time'] = dt.datetime.now().time().strftime("%I:%M:%S %p")


        self.show_stats()

    def show_stats(self):
        print('\nNames Updated!')
        print('Generator\n'+'------------------')
        print(self.ng)
        print('------------------')
        print('\n' + 'Name Stats' +
              '\n' + 'Stuck? ' + str(self.stuck) +
              '\n' + '--------------')
        print(self.stats['name_data'])
        print('--------------')
        print('Last Update @ ' + self.stats['time'])


    def get_names(self):

        names = pd.DataFrame(columns=['Neopet','L'])

        for name_group in self.dd.itertuples():
            for i,matrix in enumerate(self.ng_list[name_group[0]]):
                temp_names = build_names( matrix, name_group[3], stuck=self.stuck)

                names = pd.concat([names,temp_names],ignore_index=True)
        names = names.drop_duplicates().sort_values(by=['L','Neopet'])\
                     .reset_index(drop=True)
        self.names = names
        self.update_stats()
        return names

    def get_ng(self):
        # Create generator
        name_format = self.nf
        seglist = name_format.split(',')
        seg_df = pd.DataFrame({}, columns=['pieces','Lmin','Lmax','regex'])

        for seg in seglist:
            if ':' in seg:  # segment types denoted by preceeding :
                seglist2 = seg.split(':')

                if 'bit' in seglist2[0]:

                    if len(seglist2[1]) == 2:
                    # check bit length range; if not specified, set to default
                        Lmin = int(seglist2[1][0])
                        Lmax = int(seglist2[1][1])
                    else:
                        Lmin = 1
                        Lmax = 6

                    # Interpret regex if included
                    if len(seglist2[2])>0:
                        regex = r''
                        ui = seglist2[2] #user input
                        i=0
                        ilist = []
                        while i < len(ui):
                            if ui[i] =='[':

                                while ui[i] != ']':
                                    regex += ui[i]
                                    i+=1
                                regex += ']'
                            elif ui[i] == '.':
                                regex += '.'
                                ilist = ilist[:-1]
                            else:
                                if ui[i] == 'x':
                                    regex += '[a-z]'
                                elif ui[i] == 'c':
                                    regex += '[^aeiouy]'
                                elif ui[i] == 'v':
                                    regex += '[aeiouy]'

                            ilist += [i]
                            if len(ilist) > Lmin:
                                raise NameError('Cannot define more characters than minimum word segment (\'bit\') length; \nuse \'.\' as a placeholder instead.')
                            i += 1
                        

                    else:
                        regex = r'[a-z]'*Lmin + r'.'*(Lmax-Lmin)
                    seg_df.loc[len(seg_df)] = ['bit*',Lmin,Lmax,regex]

                else:
#                 'x' in seg or 'v' in seg or 'c' in seg:
                    if len(seglist2[1]) > 0:
                        L = int(seglist2[1])
                        seglist2[0] = seglist2[0]*L
                    for l in seglist2[0]:
                        if l == 'x':
                            seg_df.loc[len(seg_df)] = ['x*',1,1,r'[a-z]']
                        elif l =='v':
                            seg_df.loc[len(seg_df)] = ['v*',1,1,r'[aeiouy]']
                        elif l == 'c':
                            seg_df.loc[len(seg_df)] = ['c*',1,1,r'[^aeiouy]']
            else:
                seg_df.loc[len(seg_df)] = [seg,len(seg),len(seg),seg]


        # Update Properties
        self.Lmin = seg_df.Lmin.sum()
        self.Lmax = seg_df.Lmax.sum()
        self.ng = seg_df # name generator

        self.get_ng_list()

    def get_ng_list(self,bits=w_s):
        ng = self.ng
        if self.stuck == True:
            if ng.pieces[0] in ['x*','v*','c*','bit*'] == True:
                while ng.loc[0,'Lmin'] < 3:
                    ng.loc[0,'Lmin'] += ng.loc[1,'Lmin']
                    ng.loc[0,'Lmax'] += ng.loc[1,'Lmax']
                    ng.loc[0,'regex'] += ng.loc[1,'regex']
                    ng = ng.drop(1,axis=0).reset_index(drop=True)

                ng.loc[0,'pieces'] = 'bit'

        ng_list = []
        ng_list = [pd.DataFrame(columns=['piece','L','regex'])]

        for piece in ng.itertuples():
            piece_no = piece[0]
            piece_name = piece[1]
            Lmin = piece[2]
            Lmax = piece[3]
            abc_count = piece[4]


            if Lmin != Lmax:

                ng_list_new = []
                for j in range(len(ng_list)):
                    for i in range((Lmax-Lmin)+1):
                        ng_list_new.append(ng_list[j].copy())

                L_list = list(range(Lmin,Lmax+1))
                dots = piece[4].count('.')
                for i in range(0,len(ng_list_new),len(L_list)):
                    for l,L in enumerate(L_list):
                        regex = piece[4].replace('.','',dots)
                        regex = regex.replace('.','[a-z]')
                        ng_list_new[i+l].loc[piece_no] = ['bit*',L,regex]

                        dots -=1

                ng_list = ng_list_new


            else:
                for ng_i in ng_list:
                    ng_i.loc[piece_no] = [piece_name,Lmin,piece[4]]

        dist_data = pd.DataFrame({ 'L':list(range(self.Lmin,self.Lmax+1)) } )
        dist_data['N_matrix'] = [0]*len(dist_data)

        ng_list2 = [[]]*len(dist_data)
        for i in ng_list:
            L = i.L.sum()
            dist_data.loc[dist_data.L==L,'N_matrix'] += 1
            ng_list2[L-self.Lmin] = ng_list2[L-self.Lmin] + [i]


        self.ng_list = ng_list2

        dist_data['names_per'] = np.ceil(self.N/dist_data.N_matrix/len(dist_data)).astype(int)
        self.dd = dist_data

#ns = nameGen('bit:35:,bit:23:')


HTML Updated!


0      <link rel="stylesheet" type="text/css" href="s...
1                                                     \n
2                                 <div class="petgroup">
3                                                 \t<h2>
4                                          \t\t5 Letters
                             ...                        
323    \t<a href="http://neopets.com/~Trytwfos" style...
324    \t<a href="http://neopets.com/~Tuoiiuiq" style...
325    \t<a href="http://neopets.com/~Twiurlgy" style...
326                                               </div>
327                                                   \n
Length: 328, dtype: object